<a href="https://colab.research.google.com/github/aswinaus/Agents/blob/main/Routing_Agent_with_Semantic_Caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index -q
!pip install langchain -q
!pip install langchain_experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00


In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] =  userdata.get('OPENAI_API_KEY')

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
# Setup OpenAI Model and Embeddings used for indexing the documents
Settings.llm = OpenAI(model='gpt-4o-mini', temperature=0.2)
Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small')
Settings.chunk_size = 1024

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

Mounted at /content/drive


In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import VectorStoreIndex, SummaryIndex

In [ ]:
# In order to avoid repeated calls to LLMs we can store the documents index and load it if present else create it
PERSIST_INDEX_DIR = f"/{data_dir}/RAG/data/"
def get_index(index_name, doc_file_path):
  index = None
  if not os.path.exists(f"{PERSIST_INDEX_DIR}{index_name}/"):
    # Load the documents
    documents = SimpleDirectoryReader(input_files=[doc_file_path]).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # Store the index to disk
    index.storage_context.persist(f"{PERSIST_INDEX_DIR}{index_name}/")
  else: # Load index from disk
    storage_context = StorageContext.from_defaults(persist_dir=f"{PERSIST_INDEX_DIR}{index_name}/")
    index = load_index_from_storage(storage_context)

  return index

In [ ]:
# Load OECD guidelines documents for Transfer Pricing
docs_OECD_guidelines = SimpleDirectoryReader(f"{data_dir}/RAG/data/OECD/").load_data()
# Load OECD guidelines documents for Form990
docs_Form990_guidelines = SimpleDirectoryReader(f"{data_dir}/RAG/data/Form990/").load_data()

In [ ]:
#initialise a storage context and use that for both Vector Index and Summary Index for OECD
oecd_nodes = Settings.node_parser.get_nodes_from_documents(docs_OECD_guidelines)
form990_nodes = Settings.node_parser.get_nodes_from_documents(docs_Form990_guidelines)

oecd_storage_context = StorageContext.from_defaults()

oecd_storage_context.docstore.add_documents(oecd_nodes)
oecd_storage_context.docstore.add_documents(form990_nodes)
# Setup Vector and Summary Index from Storage Context
oecd_summary_index = SummaryIndex(oecd_nodes, storage_context=oecd_storage_context)
oecd_vector_index = VectorStoreIndex(oecd_nodes, storage_context=oecd_storage_context)

# Setup Indices.In order to avoid repeated calls to LLMs we can store the documents index and load it if present else create it
OECD_index = get_index("OECDTPGuidelines",f"{data_dir}/RAG/data/OECD/OECD_Transfer_Pricing_Guidelines.pdf")
form990_guidelines_index = get_index("Form990Guidelines",f"{data_dir}/RAG/data/Form990/Form990_Guidelines.pdf")

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
# Create the query engines
OECD_engine = OECD_index.as_query_engine(similarity_top_k=3)
form990_guidelines_engine = form990_guidelines_index.as_query_engine(similarity_top_k=3)
# Create tools for the query engines
OECD_query_tool = QueryEngineTool(
                      query_engine=OECD_engine,
                      metadata=ToolMetadata(
                          name="OECD_QueryEngineTool_2022",
                          description="Provides information about Transfer Pricing Guidelines for Organization from OECD for year 2022"
                      )
                    )

Form990_query_tool = QueryEngineTool(
                      query_engine=form990_guidelines_engine,
                      metadata=ToolMetadata(
                          name="form990_2022",
                          description="Provides information about Central Jersey Form990 filling for the year 2022"
                      )
                    )

tools = [OECD_query_tool, Form990_query_tool]

filing_engine = RouterQueryEngine(
                      selector= LLMSingleSelector.from_defaults(),
                      query_engine_tools=tools
                      )

In [ ]:
OECD_query="What documentation and approach is required for the valuation of assets in Transfer Pricing?"
Form990_query="An organization receives contributions worth of $15000 which Form within Form990 should the organization complete before filing returns to IRS"

In [ ]:
response = filing_engine.query(OECD_query)
print (response)
print("\n.------------------.\n")
response = filing_engine.query(Form990_query)
print (response)
print("\n.------------------.\n")

For the valuation of assets in transfer pricing, it is essential to follow a structured documentation approach. This includes:

1. **Functional and Economic Analysis**: Conduct a thorough analysis to ensure that the functional and economic aspects of the transactions are accurately represented. This analysis should be periodically reviewed to confirm its relevance and accuracy.

2. **Three-Tiered Documentation**: Implement a three-tiered approach that typically includes a master file, a local file, and a Country-by-Country Report. Each of these components should be updated annually, although certain elements may not change significantly from year to year.

3. **Use of Comparables**: When valuing assets, it is important to identify and use reliable comparables. This involves a systematic process that includes determining the years to be covered, understanding the controlled transactions, and selecting the most appropriate transfer pricing method based on the circumstances.

4. **Complia

In [ ]:
# Define the Summary and Vector query engines for OECD
summary_query_engine = oecd_summary_index.as_query_engine(response_mode= "tree_summarize")
vector_query_engine = oecd_vector_index.as_query_engine()

# Now Create the query engine tools from the above query engines
summary_tool = QueryEngineTool(
                query_engine=summary_query_engine,
                metadata=ToolMetadata(
                    name="OECD_Summary",
                    description="Summarizes the OECD guidelines for Transfer Pricing"
                    )
                )
vector_tool = QueryEngineTool(
                query_engine=vector_query_engine,
                metadata = ToolMetadata(
                    name="OECD_Guidelines_QA",
                    description="Retrieves answers for questions on OECD"
                    )
                )
oecd_tools = [summary_tool, vector_tool]
# Now define the Router Query Engine
oecd_engine = RouterQueryEngine(
                      selector= LLMSingleSelector.from_defaults(),
                      query_engine_tools=oecd_tools
                      )


In [ ]:
# Summarization routing
response = oecd_engine.query("Summarize the OECD guidelines for Transfer Pricing?")
print (response)

The OECD guidelines for Transfer Pricing establish a framework for multinational enterprises (MNEs) to ensure that transactions between associated entities are priced in accordance with the arm's length principle. This principle mandates that the pricing of intercompany transactions should reflect the prices that would be charged between independent parties under comparable circumstances.

Key elements of the guidelines include:

1. **Arm's Length Principle**: Transactions should be priced as if they were between unrelated parties.

2. **Functional Analysis**: A detailed assessment of the functions performed, assets used, and risks assumed by each party is essential for determining appropriate pricing.

3. **Transfer Pricing Methods**: The guidelines outline several methods for establishing arm's length prices, including Comparable Uncontrolled Price (CUP), Resale Price Method, Cost Plus Method, Transactional Net Margin Method, and Profit Split Method. The choice of method should be ba

In [ ]:
# To Confirm whether the Summary Engine was used
print (response.metadata["selector_result"])

selections=[SingleSelection(index=0, reason='The question specifically asks for a summary of the OECD guidelines for Transfer Pricing, which corresponds directly to choice 1.')]


In [ ]:
# Now route to a Question Answer engine
response = oecd_engine.query("What would be the best possible way to evaluate the Intangibles in Transfer Pricing?")
print (response)

The best possible way to evaluate intangibles in transfer pricing involves a comprehensive approach that includes several key steps:

1. **Identification of Intangibles**: Clearly define the specific intangible assets involved, such as patents, trademarks, or proprietary technologies.

2. **Functional Analysis**: Conduct a detailed analysis to understand the roles and contributions of each party in relation to the intangibles, including the functions performed, assets used, and risks assumed.

3. **Valuation Techniques**: Utilize appropriate valuation methods, which may include:
   - **Comparable Uncontrolled Price (CUP) Method**: When reliable comparables are available.
   - **Income Approach**: Estimating the present value of future economic benefits expected from the intangible.
   - **Cost Approach**: Assessing the costs incurred in developing the intangible, though this may not always reflect market value.

4. **Comparability Analysis**: Identify comparable uncontrolled transactio

In [ ]:
# Confirm that Vector Engine was used.
print (response.metadata["selector_result"])

selections=[SingleSelection(index=0, reason='The OECD guidelines for Transfer Pricing provide a framework for evaluating intangibles in transfer pricing, making it the most relevant choice.')]


In [ ]:
!pip install -U faiss-cpu sentence_transformers transformers

In [ ]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

class SemanticCaching:
    def __init__(self, json_file='cache.json'):
        # Initialize Faiss index  with Euclidean distance
        self.index =faiss.IndexFlatL2(768)  # Use IndexFlatL2 with Euclidean distance
        if self.index.is_trained:
            print('Index trained')

        # Initialize Sentence Transformer model
        self.encoder = SentenceTransformer('all-mpnet-base-v2')


        # Uncomment the following lines to use DialoGPT for question generation
        # self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
        # self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

        # Set Euclidean distance threshold
        self.euclidean_threshold = 0.3
        self.json_file = json_file
        self.load_cache()

    def load_cache(self):
        # Load cache from JSON file, creating an empty cache if the file is not found
        try:
            with open(self.json_file, 'r') as file:
                self.cache = json.load(file)
        except FileNotFoundError:
            self.cache = {'questions': [], 'embeddings': [], 'answers': [], 'response_text': []}

    def save_cache(self):
        # Save the cache to the JSON file
        with open(self.json_file, 'w') as file:
            json.dump(self.cache, file)

    def ask(self, question: str) -> str:
        # Method to retrieve an answer from the cache or generate a new one
        start_time = time.time()
        try:
            l = [question]
            embedding = self.encoder.encode(l)

            # Search for the nearest neighbor in the index
            D, I = self.index.search(embedding, 1)

            if D[0] >= 0:
                if I[0][0] != -1 and D[0][0] <= self.euclidean_threshold:
                    row_id = int(I[0][0])
                    print(f'Found cache in row: {row_id} with score {1 - D[0][0]}') #score inversed to show similarity
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    print(f"Time taken: {elapsed_time} seconds")
                    return self.cache['response_text'][row_id]

            # Handle the case when there are not enough results or Euclidean distance is not met
            answer, response_text = self.generate_answer(question)

            self.cache['questions'].append(question)
            self.cache['embeddings'].append(embedding[0].tolist())
            self.cache['answers'].append(answer)
            self.cache['response_text'].append(response_text)

            self.index.add(embedding)
            self.save_cache()
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Time taken: {elapsed_time} seconds")

            return response_text
        except Exception as e:
            raise RuntimeError(f"Error during 'ask' method: {e}")

    def generate_answer(self, question: str) -> str:
        # Method to generate an answer using a separate function (make_prediction in this case)
        try:
            result = oecd_engine.query(question)
            #response_text = result['data']['response_text']
            response_text=result

            return result, response_text
        except Exception as e:
            raise RuntimeError(f"Error during 'generate_answer' method: {e}")

In [ ]:
cache = SemanticCaching()

In [ ]:
import markdown
from IPython.display import Markdown

def print_gpt_markdown(markdown_text):
    display(Markdown(markdown_text))

In [ ]:
# Now Cache the request and response using Semantic Caching wherein the question is routed to a Question Answer engine
response = cache.ask("What would be the best possible way to evaluate the Intangibles in Transfer Pricing?")
print(response)


Found cache in row: 0 with score 1.0
Time taken: 0.13820385932922363 seconds
The best possible way to evaluate intangibles in transfer pricing involves a comprehensive approach that adheres to the arm's length principle. This includes conducting a thorough functional analysis to assess the contributions of each party involved in the transaction, focusing on the functions performed, assets used, and risks assumed.

Key steps in this evaluation process include:

1. **Functional Analysis**: Identify and assess the specific contributions made by each entity regarding the development, enhancement, maintenance, protection, and exploitation of the intangible assets.

2. **Identification of Intangibles**: Clearly define the intangible assets involved, such as patents, trademarks, or proprietary technology, and understand their unique characteristics.

3. **Valuation Techniques**: Utilize appropriate valuation methods, including:
   - **Comparable Uncontrolled Price (CUP) Method**: When reliabl

In [ ]:
# Querying the same agent with a different question but with same semantic meaning
response = cache.ask("How can Intangibles be effectively evaluated in Transfer Pricing?")
print(response)


Found cache in row: 0 with score 0.8939292430877686
Time taken: 0.11674809455871582 seconds
The best possible way to evaluate intangibles in transfer pricing involves a comprehensive approach that adheres to the arm's length principle. This includes conducting a thorough functional analysis to assess the contributions of each party involved in the transaction, focusing on the functions performed, assets used, and risks assumed.

Key steps in this evaluation process include:

1. **Functional Analysis**: Identify and assess the specific contributions made by each entity regarding the development, enhancement, maintenance, protection, and exploitation of the intangible assets.

2. **Identification of Intangibles**: Clearly define the intangible assets involved, such as patents, trademarks, or proprietary technology, and understand their unique characteristics.

3. **Valuation Techniques**: Utilize appropriate valuation methods, including:
   - **Comparable Uncontrolled Price (CUP) Method*

In [ ]:
# Querying the same agent with a different question but with same semantic meaning
response = cache.ask("What methods are recommended for assessing Intangibles in Transfer Pricing?")
print(response)

Found cache in row: 0 with score 0.8853908404707909
Time taken: 0.1242213249206543 seconds
The best possible way to evaluate intangibles in transfer pricing involves a comprehensive approach that adheres to the arm's length principle. This includes conducting a thorough functional analysis to assess the contributions of each party involved in the transaction, focusing on the functions performed, assets used, and risks assumed.

Key steps in this evaluation process include:

1. **Functional Analysis**: Identify and assess the specific contributions made by each entity regarding the development, enhancement, maintenance, protection, and exploitation of the intangible assets.

2. **Identification of Intangibles**: Clearly define the intangible assets involved, such as patents, trademarks, or proprietary technology, and understand their unique characteristics.

3. **Valuation Techniques**: Utilize appropriate valuation methods, including:
   - **Comparable Uncontrolled Price (CUP) Method**

In [ ]:
# Querying the same agent with a different question but with same semantic meaning
response = cache.ask("What strategies are considered most effective for evaluating Intangibles in Transfer Pricing?")
print(response)

Found cache in row: 0 with score 0.8500476181507111
Time taken: 0.1260666847229004 seconds
The best possible way to evaluate intangibles in transfer pricing involves a comprehensive approach that adheres to the arm's length principle. This includes conducting a thorough functional analysis to assess the contributions of each party involved in the transaction, focusing on the functions performed, assets used, and risks assumed.

Key steps in this evaluation process include:

1. **Functional Analysis**: Identify and assess the specific contributions made by each entity regarding the development, enhancement, maintenance, protection, and exploitation of the intangible assets.

2. **Identification of Intangibles**: Clearly define the intangible assets involved, such as patents, trademarks, or proprietary technology, and understand their unique characteristics.

3. **Valuation Techniques**: Utilize appropriate valuation methods, including:
   - **Comparable Uncontrolled Price (CUP) Method**

In [ ]:
# Querying the same agent with a different question but with same semantic meaning
response = cache.ask("How should Intangibles be properly assessed in the context of Transfer Pricing?")
print(response)

Found cache in row: 0 with score 0.8263822793960571
Time taken: 0.21420955657958984 seconds
The best possible way to evaluate intangibles in transfer pricing involves a comprehensive approach that adheres to the arm's length principle. This includes conducting a thorough functional analysis to assess the contributions of each party involved in the transaction, focusing on the functions performed, assets used, and risks assumed.

Key steps in this evaluation process include:

1. **Functional Analysis**: Identify and assess the specific contributions made by each entity regarding the development, enhancement, maintenance, protection, and exploitation of the intangible assets.

2. **Identification of Intangibles**: Clearly define the intangible assets involved, such as patents, trademarks, or proprietary technology, and understand their unique characteristics.

3. **Valuation Techniques**: Utilize appropriate valuation methods, including:
   - **Comparable Uncontrolled Price (CUP) Method*

In [ ]:
# Querying the same agent with a different question but with same semantic meaning
response = cache.ask("What are the most reliable approaches for evaluating Intangibles in Transfer Pricing?")
print(response)

Found cache in row: 0 with score 0.9031984657049179
Time taken: 0.12486481666564941 seconds
The best possible way to evaluate intangibles in transfer pricing involves a comprehensive approach that adheres to the arm's length principle. This includes conducting a thorough functional analysis to assess the contributions of each party involved in the transaction, focusing on the functions performed, assets used, and risks assumed.

Key steps in this evaluation process include:

1. **Functional Analysis**: Identify and assess the specific contributions made by each entity regarding the development, enhancement, maintenance, protection, and exploitation of the intangible assets.

2. **Identification of Intangibles**: Clearly define the intangible assets involved, such as patents, trademarks, or proprietary technology, and understand their unique characteristics.

3. **Valuation Techniques**: Utilize appropriate valuation methods, including:
   - **Comparable Uncontrolled Price (CUP) Method*